In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
from ydata_profiling import ProfileReport

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import warnings
import os
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(r"data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df = df.drop("Unnamed: 32", axis = 1)

In [4]:
df.diagnosis.value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [5]:
df["diagnosis"] = LabelEncoder().fit_transform(df["diagnosis"])
df.diagnosis.value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

In [6]:
import scipy.stats as stats
df[df.columns[2:]] = df[df.columns[2:]].apply(stats.zscore)

In [7]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,...,1.886690,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015
1,842517,1,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,...,1.805927,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190
2,84300903,1,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,...,1.511870,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391
3,84348301,1,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,...,-0.281464,0.133984,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010
4,84358402,1,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,...,1.298575,-1.466770,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100


In [8]:
corrValues = df.corrwith(df["diagnosis"])
corrValues.sort_values()

smoothness_se             -0.067016
fractal_dimension_mean    -0.012838
texture_se                -0.008303
symmetry_se               -0.006522
id                         0.039769
fractal_dimension_se       0.077972
concavity_se               0.253730
compactness_se             0.292999
fractal_dimension_worst    0.323872
symmetry_mean              0.330499
smoothness_mean            0.358560
concave points_se          0.408042
texture_mean               0.415185
symmetry_worst             0.416294
smoothness_worst           0.421465
texture_worst              0.456903
area_se                    0.548236
perimeter_se               0.556141
radius_se                  0.567134
compactness_worst          0.590998
compactness_mean           0.596534
concavity_worst            0.659610
concavity_mean             0.696360
area_mean                  0.708984
radius_mean                0.730029
area_worst                 0.733825
perimeter_mean             0.742636
radius_worst               0

In [9]:
features = ["area_se",
            "perimeter_se",
            "radius_se",
            "compactness_worst",
            "compactness_mean",
            "concavity_worst",
            "concavity_mean",
            "area_mean",
            "radius_mean",
            "area_worst",
            "perimeter_mean",
            "radius_worst",
            "concave points_mean",
            "perimeter_worst",
            "concave points_worst",
            "diagnosis"
           ]

In [10]:
new_df = df[features]
new_df

,area_se,perimeter_se,radius_se,compactness_worst,compactness_mean,concavity_worst,concavity_mean,area_mean,radius_mean,area_worst,perimeter_mean,radius_worst,concave points_mean,perimeter_worst,concave points_worst,diagnosis
0,2.487578,2.833031,2.489734,2.616665,3.283515,2.109526,2.652874,0.984375,1.097064,2.001237,1.269934,1.886690,2.532475,2.303601,2.296076,1
1,0.742402,0.263327,0.499255,-0.430444,-0.487072,-0.146749,-0.023846,1.908708,1.829821,1.890489,1.685955,1.805927,0.548144,1.535126,1.087084,1
2,1.181336,0.850928,1.228676,1.082932,1.052926,0.854974,1.363478,1.558884,1.579888,1.456285,1.566503,1.511870,2.037231,1.347475,1.955000,1
3,-0.288378,0.286593,0.326373,3.893397,3.402909,1.989588,1.915897,-0.764464,-0.768909,-0.550021,-0.592687,-0.281464,1.451707,-0.249939,2.175786,1
4,1.190357,1.273189,1.270543,-0.313395,0.539340,0.613179,1.371011,1.826229,1.750297,1.220724,1.776573,1.298575,1.428493,1.338539,0.729259,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,2.604187,2.379583,2.782080,-0.273318,0.219060,0.664512,1.947285,2.343856,2.110995,2.015301,2.060786,1.901185,2.320965,1.752563,1.629151,1
565,1.291565,1.156857,1.300499,-0.394820,-0.017833,0.236573,0.693043,1.723842,1.704854,1.494959,1.615931,1.536720,1.263669,1.421940,0.733827,1
566,0.180698,0.276693,0.184892,0.350735,-0.038680,0.326767,0.046588,0.577953,0.702284,0.427906,0.672676,0.561361,0.105777,0.579001,0.414069,1
567,1.009503,1.438530,1.157935,3.904848,3.272144,3.197605,3.296944,1.735218,1.838341,1.653171,1.982524,1.961239,2.658866,2.303601,2.289985,1


In [11]:
new_df.diagnosis.value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

In [12]:
X = new_df.drop(["diagnosis"], axis=1)
Y = new_df["diagnosis"]
X.head()

,area_se,perimeter_se,radius_se,compactness_worst,compactness_mean,concavity_worst,concavity_mean,area_mean,radius_mean,area_worst,perimeter_mean,radius_worst,concave points_mean,perimeter_worst,concave points_worst
0,2.487578,2.833031,2.489734,2.616665,3.283515,2.109526,2.652874,0.984375,1.097064,2.001237,1.269934,1.886690,2.532475,2.303601,2.296076
1,0.742402,0.263327,0.499255,-0.430444,-0.487072,-0.146749,-0.023846,1.908708,1.829821,1.890489,1.685955,1.805927,0.548144,1.535126,1.087084
2,1.181336,0.850928,1.228676,1.082932,1.052926,0.854974,1.363478,1.558884,1.579888,1.456285,1.566503,1.511870,2.037231,1.347475,1.955000
3,-0.288378,0.286593,0.326373,3.893397,3.402909,1.989588,1.915897,-0.764464,-0.768909,-0.550021,-0.592687,-0.281464,1.451707,-0.249939,2.175786
4,1.190357,1.273189,1.270543,-0.313395,0.539340,0.613179,1.371011,1.826229,1.750297,1.220724,1.776573,1.298575,1.428493,1.338539,0.729259


In [13]:
# "XG Boost": XGBClassifier(),
classifiers = {
    "Random Forest": RandomForestClassifier(),
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC()
}

In [14]:
skf = StratifiedKFold(n_splits=10, random_state=None, shuffle = False)

for train_index, test_index in skf.split(X, Y):
    Xtrain, Xtest = X.iloc[train_index], X.iloc[test_index]
    Ytrain, Ytest = Y.iloc[train_index], Y.iloc[test_index]

Xtrain = Xtrain.values
Xtest = Xtest.values
Ytrain = Ytrain.values
Ytest = Ytest.values

In [15]:
for key, classifier in classifiers.items():
    classifier.fit(Xtrain, Ytrain)
    Ypred = classifier.predict(Xtest)
    acc_score = accuracy_score(Ypred, Ytest)
    print("Classifiers: ", classifier.__class__.__name__, "has a accuracy score of", acc_score)

Classifiers:  RandomForestClassifier has a accuracy score of 0.9821428571428571
Classifiers:  LogisticRegression has a accuracy score of 0.9642857142857143
Classifiers:  KNeighborsClassifier has a accuracy score of 0.9642857142857143
Classifiers:  SVC has a accuracy score of 0.9464285714285714
